In [ ]:
from pymycobot.mycobot import MyCobot
from pymycobot import PI_PORT, PI_BAUD
import time
import csv
import math
import numpy as np
from sympy import symbols, cos, sin, pi, simplify
from sympy.matrices import Matrix

In [ ]:
# Initialize MyCobot
# mc = MyCobot(PI_PORT, PI_BAUD)

#Global variables
speed = 30 # Speed for moving servos
sleepTime = 5 # Sleep time after moving servos
origin = [0, 0, 0, 0, 0, 0] # Origin position for servos
q1, q2, q3, q4, q5, q6 = symbols('q1 q2 q3 q4 q5 q6') # Define symbolic variables for joint angles
right = pi/2

In [ ]:
# Error correction settings
max_retries = 3  # Number of times to retry sending angles if they don't match
angle_tolerance = 1.0  # Degrees tolerance for angle comparison
coord_tolerance = 5.0  # mm (or units returned by get_coords) tolerance for coordinate comparison

In [ ]:
# Generate Empty DH table for MechArm
# DH table rows: [a, alpha, d, offset, offset]
# a      : link length (mm)
# alpha  : link twist (radians)
# d      : link offset (mm)
# offset : constant offset added to the joint variable (radians)
# q      : symbolic joint variable (q1..q6)
dh_table = [
    [0,     0,      95,    0,      q1],   # joint 1
    [0,    -right,   0,   -right,  q2],   # joint 2
    [90,    0,       0,    0,      q3],   # joint 3
    [10,    right,  95,    0,      q4],   # joint 4
    [0,     right,   0,    0,      q5],   # joint 5
    [0,    -right,  50,    0,      q6]    # joint 6
]

In [ ]:
# Reset servos to origin position
def reset_servos():
    mc.power_off()
    time.sleep(sleepTime)
    mc.power_on()
    time.sleep(sleepTime)
    mc.send_angles(origin, speed)
    time.sleep(sleepTime)

# Get current servo positions
def get_current_positions():
    Coords = mc.get_coords()
    angles = mc.get_angles()
    return angles, Coords

def get_transformation_matrix(a, alpha, d, offset, theta):
    return Matrix([
        [cos(theta + offset), -sin(theta + offset)*cos(alpha), sin(theta + offset)*sin(alpha), a*cos(theta + offset)],
        [sin(theta + offset), cos(theta + offset)*cos(alpha), -cos(theta + offset)*sin(alpha), a*sin(theta + offset)],
        [0, sin(alpha), cos(alpha), d],
        [0, 0, 0, 1]
    ])

def overall_transformation(dh_table):
    T = Matrix(np.identity(4))
    for i in range(len(dh_table)):
        a, alpha, d, offset, theta = dh_table[i]
        T_i = get_transformation_matrix(a, alpha, d, offset, theta)
        T = T * T_i
    return simplify(T)

In [ ]:
# ask user for joint angles
joint_angles = input("Enter joint angles (q1, q2, q3, q4, q5, q6) separated by commas: ")
joint_angles = [np.radians(float(angle)) for angle in joint_angles.split(",")]
# Set joint angles to symbolic variables
q1_val, q2_val, q3_val, q4_val, q5_val, q6_val = joint_angles
# Calculate overall transformation matrix
T = overall_transformation(dh_table)
# Substitute joint angles into transformation matrix
T_substituted = T.subs({q1: q1_val, q2: q2_val, q3: q3_val, q4: q4_val, q5: q5_val, q6: q6_val})
end_effector_position = T_substituted[:3, 3]